In [14]:
%matplotlib inline
import random
import torch
import numpy as np

In [7]:
def synthetic_data(w, b, num_examples):
  X = torch.normal(0, 1, (num_examples, len(w)))
  y = torch.matmul(X, w) + b
  y += torch.normal(0, 0.1, y.shape)
  return X, y.reshape(len(y), -1)

true_w = torch.tensor([2, -3.4])
true_b = 4.2
X_in, y_label = synthetic_data(true_w, true_b, 1000)

In [8]:
print(y_label.shape)

torch.Size([1000, 1])


In [9]:
print(X_in[0])
print(y_label[0])

tensor([-0.1346,  0.8612])
tensor([0.9914])


In [12]:
def data_iter(batch_size, X, y):
  indices = list(range(len(X)))
  random.shuffle(indices)
  for i in range(0, len(X), batch_size):
    batch_indices = np.array(indices[i : min(i+batch_size, len(X))])
    yield X[batch_indices], y[batch_indices]

# **Train**

In [50]:
w = torch.normal(0, 0.01 , size=(2 , 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [32]:
def linreg(X, w, b):
  return torch.matmul(X, w) + b

In [33]:
def squared_loss(y_hat, y):
  return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [45]:
def SGD(params, lr, batch_size):
  with torch.no_grad(): # set up for reduce bo nho
    for param in params:
      param -= lr * param.grad / batch_size
      param.grad.zero_() # must set_up 

In [35]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
batch_size = 10

In [53]:
for epoch in range(num_epochs):
  for X, y in data_iter(batch_size, X_in, y_label):
    l = loss(net(X, w, b), y)
    l.sum().backward()
    SGD([w, b], lr, batch_size)
  with torch.no_grad():
    train_l = loss(net(X_in, w, b), y_label)
    print(train_l.shape)
    print(f'epoch {epoch + 1}, loss: {float(train_l.mean()):.5f}')

torch.Size([1000, 1])
epoch 1, loss: 0.00525
torch.Size([1000, 1])
epoch 2, loss: 0.00524
torch.Size([1000, 1])
epoch 3, loss: 0.00523


In [55]:
print(w)
print(b)

tensor([[ 1.9982],
        [-3.3998]], requires_grad=True)
tensor([4.1959], requires_grad=True)
